In [3]:
!pip install git+https://github.com/zc277584121/graphrag.git

  Cloning https://github.com/zc277584121/graphrag.git to /tmp/pip-req-build-lndckr9m
  Running command git clone --filter=blob:none --quiet https://github.com/zc277584121/graphrag.git /tmp/pip-req-build-lndckr9m
  Resolved https://github.com/zc277584121/graphrag.git to commit c7263931eb9b3b6c7cd72fc11e4a6d4b11e6d48b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import nest_asyncio
nest_asyncio.apply()

import os
import urllib.request

In [3]:
index_root = os.path.join(os.getcwd(), 'graphrag_index')
os.makedirs(os.path.join(index_root, 'input'), exist_ok=True)

# # with open('/home/nuc11extreme-012/Downloads/extracted_texts', 'r+', encoding='utf-8') as file:
#     lines = file.readlines()
#     file.seek(0)
    

In [66]:
!python3 -m graphrag.index --init --root ./graphrag_index 

Initializing project at ./graphrag_index
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File Indexer 
"/home/nuc11extreme-012/anaconda3/lib/python3.12/site-packages/graphrag/index/__
main__.py", line 76, in <module>
    index_cli( 
  File Indexer 
"/home/nuc11extreme-012/anaconda3/lib/python3.12/site-packages/graphrag/index/cl
i.py", line 90, in index_cli
    _initialize_project_at(root, progress_reporter)
  File Indexer 
"/home/nuc11extreme-012/anaconda3/lib/python3.12/site-packages/graphrag/index/cl
i.py", line 184, in _initialize_project_at
    raise ValueError(msg)
ValueError: Project already initialized at graphrag_index
⠋ GraphRAG Indexer 

In [67]:
!python3 -m graphrag.index --root ./graphrag_index

🚀 Reading settings from graphrag_index/settings.yaml
⠹ GraphRAG Indexer 
⠹ GraphRAG Indexer e.text) - 16 files loaded (12 filtered)  100%  0…
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
└── create_base_text_units
⠸ GraphRAG Indexer ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   6% -:--:-- 0:00:00
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠼ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtered)  100%  0…
⠼ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 16 files loaded (12 filtere

In [4]:
import pandas as pd
import tiktoken
from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    # read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores import MilvusVectorStore

In [5]:
output_dir = os.path.join(index_root, "output")
subdirs = [os.path.join(output_dir, d) for d in os.listdir(output_dir)]
latest_subdir = max(subdirs, key=os.path.getmtime)  # Get latest output directory
INPUT_DIR = os.path.join(latest_subdir, "artifacts")

In [6]:
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [7]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

In [8]:
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
description_embedding_store = MilvusVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(uri="http://localhost:19530") # For Milvus docker service
# description_embedding_store.connect(uri="./milvus.db") # For Milvus Lite
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)
print(f"Entity count: {len(entity_df)}")


Entity count: 332


In [9]:
entity_df.head(10)

,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,"""SCALING BITCOIN WORKSHOP""","""EVENT""","The ""Scaling Bitcoin Workshop"" is an event ded...","2840fdbebae6832a91b2495bce86cfdd,4987e356a8b11...",1,5,0,b45241d70f0e43fca764df95b2b81f77,5,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,"""MONTREAL 2015""","""EVENT""","""Montreal 2015"" refers to a specific instance ...","2840fdbebae6832a91b2495bce86cfdd,53a62417b7faf...",2,8,1,4119fd06010c494caa07f439b333f4c5,8,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,"""BASE58 ASSOCIATION""","""ORGANIZATION""",The Base58 Association is an organization dedi...,"03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",1,8,2,d3835bf3dda84ead99deadbeac5d0d7d,8,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,"""LOCAL SUPPORTING ORGANIZATIONS""","""ORGANIZATION""",Local Supporting Organizations are entities th...,"22125aae0144999bdf83d729041f9c85,2840fdbebae68...",1,2,3,077d2820ae1845bcbb1803379a3d1eae,2,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"""BITCOIN NETWORK""","""CONCEPT""","""Bitcoin Network refers to the decentralized n...",53a62417b7faf84385e8e286eb93fd77,1,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0
5,0,"""INDUSTRIAL ACTION""","""EVENT""","""Industrial action refers to activities undert...","03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",None,0,5,19a7f254a5d64566ab5cc15472df02de,0,None,19a7f254a5d64566ab5cc15472df02de,0,0
6,0,"""SUBCONTRACTOR""","""ROLE""","""A subcontractor is a person or business that ...","03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",None,0,6,e7ffaee9d31d4d3c96e04f911d0a8f9e,0,None,e7ffaee9d31d4d3c96e04f911d0a8f9e,0,0
7,0,"""TERMINATION""","""EVENT""","""Termination refers to the act of ending a con...","03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",None,0,7,f7e11b0e297a44a896dc67928368f600,0,None,f7e11b0e297a44a896dc67928368f600,0,0
8,0,"""LEGAL CASE""","""EVENT""","""A legal case is a dispute between parties tha...","03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",None,0,8,1fd3fa8bb5a2408790042ab9573779ee,0,None,1fd3fa8bb5a2408790042ab9573779ee,0,0
9,0,"""COMPETITION LAWS""","""EVENT""","""Competition laws are regulations that promote...","03ae93602ffd69e023cf57e47d4e1288,132c1bbda79e0...",None,0,9,27f9fbe6ad8c4a8b9acee0d3596ed57c,0,None,27f9fbe6ad8c4a8b9acee0d3596ed57c,0,0


In [10]:

relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

In [11]:
print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 185


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""SCALING BITCOIN WORKSHOP""","""MONTREAL 2015""",2.0,"The ""Scaling Bitcoin Workshop"" is a series of ...","[2840fdbebae6832a91b2495bce86cfdd, 53a62417b7f...",66cdf168f36d4a57a505028c97dc06e0,0,5,8,13
1,"""SCALING BITCOIN WORKSHOP""","""BASE58 ASSOCIATION""",4.0,The Base58 Association is directly involved in...,"[2840fdbebae6832a91b2495bce86cfdd, 4987e356a8b...",38f51478f41f48db9bee570859b6f43e,1,5,8,13
2,"""SCALING BITCOIN WORKSHOP""","""BITCOIN NETWORK""",1.0,"""The Scaling Bitcoin Workshop discusses topics...",[53a62417b7faf84385e8e286eb93fd77],896d2a51e8de47de85ba8ced108c3d53,2,5,1,6
3,"""SCALING BITCOIN WORKSHOP""","""TEL AVIV 2019""",1.0,"""The Scaling Bitcoin Workshop is specifically ...",[7993fa7f98e78d35739f518698e8c1a6],14555b518e954637b83aa762dc03164e,3,5,3,8
4,"""SCALING BITCOIN WORKSHOP""","""STANFORD 2017""",1.0,"""Stanford 2017 is a specific event under the b...",[69b427c7cffd4560f7d69a328e62f581],b1f6164116d44fe8b8f135d7f65b9e58,4,5,6,11


In [12]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 30


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# Andrew Poelstra and libsecp256k1 in Bitcoin ...,1,8.5,Andrew Poelstra and libsecp256k1 in Bitcoin De...,The impact severity rating is high due to the ...,"The community centers around Andrew Poelstra, ...",[{'explanation': 'Andrew Poelstra is a promine...,"{\n ""title"": ""Andrew Poelstra and libsecp25...",dbf8aedb-27d3-4949-8135-9259e1b19f4b
1,11,# Greg Maxwell and BIP150 in Bitcoin Community...,1,7.5,Greg Maxwell and BIP150 in Bitcoin Community,The impact severity rating is high due to the ...,"The community centers around Greg Maxwell, a p...",[{'explanation': 'Greg Maxwell is a notable fi...,"{\n ""title"": ""Greg Maxwell and BIP150 in Bi...",ead0c473-fb22-4adf-b740-6abe7005dccb
2,12,# Bitcoin SPV and BIP151 Community\n\nThe comm...,1,7.5,Bitcoin SPV and BIP151 Community,The impact severity rating is high due to the ...,The community focuses on the integration of SP...,"[{'explanation': 'SPV, or Simplified Payment V...","{\n ""title"": ""Bitcoin SPV and BIP151 Commun...",b9e7d0d6-99c4-4590-8f4c-5ff6eb5eabe3
3,13,# Israeli Bitcoin Association and Meni Rosenfe...,1,6.0,Israeli Bitcoin Association and Meni Rosenfeld,The impact severity rating is moderate due to ...,The community centers around the Israeli Bitco...,[{'explanation': 'The Israeli Bitcoin Associat...,"{\n ""title"": ""Israeli Bitcoin Association a...",65dad89b-7ace-4c79-9c6b-fcd56e6cb2ac
4,14,# Segregated Witness and UTXO Set in Bitcoin\n...,1,7.5,Segregated Witness and UTXO Set in Bitcoin,The impact severity rating is high due to the ...,The community centers around the Segregated Wi...,[{'explanation': 'Segregated Witness is a prop...,"{\n ""title"": ""Segregated Witness and UTXO S...",da362a95-a499-49af-807f-d6e186b895fb


In [13]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 43


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,53a62417b7faf84385e8e286eb93fd77,Scaling Bitcoin Workshops - Transcripts\nhttp:...,634,[1769468b6a425b7423c7d2ca21419f6f],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[66cdf168f36d4a57a505028c97dc06e0, 38f51478f41..."
1,fb4b627ab5fb0d77b4bc1a3d4d1d591d,Scaling Bitcoin Workshops - Transcripts\nhttp:...,1200,[1b9df08806418f3c9687850e134d64ec],"[4119fd06010c494caa07f439b333f4c5, 19a7f254a5d...","[c8b2408617804483b620e1a6691ac90d, a5e0d1644eb..."
2,03ae93602ffd69e023cf57e47d4e1288,at the gap. The ideal gap will increase as mi...,400,[1b9df08806418f3c9687850e134d64ec],"[d3835bf3dda84ead99deadbeac5d0d7d, 19a7f254a5d...","[70634e10a5e845aa8c6a32fe7e8eb2b2, c2d48b75af6..."
3,2840fdbebae6832a91b2495bce86cfdd,Scaling Bitcoin Workshops - Transcripts\nhttp:...,968,[1c55093dde8bc0d7c80681c21036f951],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[66cdf168f36d4a57a505028c97dc06e0, 38f51478f41..."
4,1bca4bfdcc63041be24ad99338b8a024,Scaling Bitcoin Workshops - Transcripts\nhttp:...,1200,[215c06498201e08a9832381be4731799],"[19a7f254a5d64566ab5cc15472df02de, e7ffaee9d31...","[60c58026b2764b40adffca6eaa31d6d9, b0dd60e11da..."


In [14]:
api_key = os.environ["OPENAI_API_KEY"]  # Your OpenAI API key
llm_model = "o3-mini"  # Or gpt-4-turbo-preview
embedding_model = "text-embedding-3-small"

In [15]:
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,
    # max_retries=20,
)
token_encoder = tiktoken.get_encoding("cl100k_base")
text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [16]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=None, #covariates,#todo
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [ ]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    # "max_tokens": 12_000,  # change this based on the token limit you have on your model 
}

In [ ]:
llm_params = {
    # "max_tokens": 2_000,  # change this based on thhe model
    # "temperature": 0.0, #empty for o3 mini only
}

In [19]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [20]:
result = search_engine.search("how do you incentivize miners?")
print(result.response)

Below is an overview of how miner incentives are designed and why they’re so crucial to Bitcoin’s network:

## Reward Mechanisms and Economic Alignment

Miners are primarily motivated through a dual reward system. First, there are block rewards—newly minted bitcoins awarded when a miner successfully validates a new block. Second, miners receive transaction fees, which users attach to transactions to help ensure they are processed quickly. Together, these rewards form the backbone of the incentive structure that aligns a miner’s economic interests with the network’s security and efficiency [Data: Entities (38, 18); Relationships (118)].

## Incentive Structures and Protocols

To further encourage the right behaviors, the Bitcoin protocol is built with incentive structures designed to promote timely block propagation and adherence to consensus rules. For example, a miner not only earns rewards by adding a block but also benefits from rules that discourage selfish mining, orphan block cre

In [22]:
result = search_engine.search("What is the fee threshold for inclusion? ")
print(result.response)


The fee threshold for inclusion is defined as the minimum fee rate that a transaction must pay in order to be included in a block. In other words, when considering a block, the fee threshold is set to match the smallest fee rate present in that block. This minimal fee rate is used as a measure of the network load: if users need to pay a higher fee to have their transactions included, it indicates that the network is experiencing high demand and congestion [Data: Sources (14)].

This approach to determining the fee threshold has a dual purpose. It not only helps in establishing a concrete metric for deciding transaction inclusion but also acts as a dynamic indicator of network efficiency. When the threshold rises, it reflects that transactions competing for inclusion are forced to offer higher fees, thereby signaling a more heavily loaded network [Data: Sources (14), Relationships (14)].


In [23]:
result = search_engine.search("**Question:**  \nGiven a starting fee (S), a maximum fee (M), and a maximum of (B) blocks to reach that maximum, what formula computes the fee at block height (H) so that it increases exponentially from (S) up to (M) over (B) blocks?")
print(result.response)

Below is one commonly used formula for exponential interpolation between a starting fee S and a maximum fee M over B blocks. The idea is that when H = 0, the fee starts at S, and when H = B, the fee reaches M. We first define the growth factor r such that

  S · rᴮ = M

Solving for r gives

  r = (M/S)^(1/B)

Then, for any block height H (where 0 ≤ H ≤ B), the fee (F) can be computed as

  F(H) = S · rᴴ = S · (M/S)^(H/B)

This formula ensures that the fee increases exponentially from S to M as H goes from 0 to B.

It is important to note that if H exceeds B, you might want to clamp the value at M or continue with the same growth factor, depending on your application. This exponential progression is particularly useful in scenarios where fees need to adjust gradually with network conditions, ensuring a smooth transition from a lower to a higher fee structure [Data: Bitcoin Community and Its Ecosystem (record ids: 9); Entities (record ids: 127, 43) +more].
